In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [58]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [21]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [27]:
n_train_df = pd.read_csv('n_train.csv')
n_test_df = pd.read_csv('n_test.csv')

In [29]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
n_main_attrs = [c for c in n_train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(n_main_attrs))

Main features: 260


In [36]:
diff_cols = ['contest-pevpr-sfc-gauss-14d__pevpr', 'year', 'nmme0-tmp2m-34w__cancm30']

In [56]:
remain_cols = [c for c in n_main_attrs if c not in main_attrs and c not in diff_cols]
len(remain_cols)

198

In [38]:
def train_model_by_X_y(_X, _y, iterations=5000):
    skf = KFold(n_splits=5, random_state=2048, shuffle=True)
    skf.get_n_splits(_X, _y)

    train_index_list = []
    test_index_list = []
    train_x_list = []
    val_x_list = []
    train_y_list = []
    val_y_list = []

    for i, (train_index, test_index) in enumerate(skf.split(_X, _y)):
        train_index_list.append(train_index)
        test_index_list.append(test_index)

        train_x_list.append(_X[train_index, :])
        val_x_list.append(_X[test_index, :])
        train_y_list.append(_y[train_index])
        val_y_list.append(_y[test_index])    
   

    cat_models = []
    for i in range(5):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i],
        clf = CatBoostRegressor(iterations=iterations, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=5000, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models


In [39]:
def cal_train_pred(models, _X):
    preds = []
    for model in models:
        preds.append(model.predict(_X))
    s = 0
    for pred in preds:
        s += pred
    s = s/5
    return s

In [40]:
number_features = np.random.randint(100, 202, 5)
generate_feats = []
list_features = []
for i, num in tqdm(enumerate(number_features)):
    list_feat = np.random.randint(0, len(remain_cols), num)
    cols = []
    for c in list_feat:
        cols.append(remain_cols[c])
    cols.append('contest-wind-h500-14d__wind-hgt-500')
    cols = list(set(cols))
    list_features.append(cols)
    X = n_train_df[cols].values
    y = np.array(n_train_df[target[0]].tolist())
    cat_models = train_model_by_X_y(X, y)
    generate_feats.append(f"col_{i}")
    y_pred = cal_train_pred(cat_models, X)
    train_df[f"col_{i}"] = y_pred
    y_pred_test = cal_train_pred(cat_models, n_test_df[cols].values)
    test_df[f"col_{i}"] = y_pred_test


0it [00:00, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2412643	test: 9.2200410	best: 9.2200410 (0)	total: 8.17ms	remaining: 40.8s
4999:	learn: 0.4021556	test: 0.4388699	best: 0.4388699 (4999)	total: 27.4s	remaining: 0us
bestTest = 0.4388699426
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2327219	test: 9.2497618	best: 9.2497618 (0)	total: 6.45ms	remaining: 32.3s
4999:	learn: 0.3997602	test: 0.4368011	best: 0.4368011 (4999)	total: 27.6s	remaining: 0us
bestTest = 0.4368010784
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2294690	test: 9.2626367	best: 9.2626367 (0)	total: 6.9ms	remaining: 34.5s
4999:	learn: 0.4008624	test: 0.4348661	best: 0.4348661 (4999)	total: 27.8s	remaining: 0us
bestTest = 0.4348661472
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2364614	test: 9.2328684	best: 9.2328684 (0)	total: 8.79ms	remaining: 43.9s
4999:	learn: 0.3997718	test: 0.4361639	best: 0.4361639 (4999)	total: 28.1s	remaining: 0us
bestTest = 0.4361639228
bestIteration 

1it [03:26, 206.41s/it]

Learning rate set to 0.072699
0:	learn: 9.2430576	test: 9.2218640	best: 9.2218640 (0)	total: 13.8ms	remaining: 1m 8s
4999:	learn: 0.3934127	test: 0.4275707	best: 0.4275707 (4999)	total: 29.2s	remaining: 0us
bestTest = 0.4275706869
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2343072	test: 9.2515312	best: 9.2515312 (0)	total: 13ms	remaining: 1m 5s
4999:	learn: 0.3930593	test: 0.4278512	best: 0.4278512 (4999)	total: 29.6s	remaining: 0us
bestTest = 0.4278511723
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2303827	test: 9.2642715	best: 9.2642715 (0)	total: 8.13ms	remaining: 40.7s
4999:	learn: 0.3924475	test: 0.4252917	best: 0.4252917 (4999)	total: 29.5s	remaining: 0us
bestTest = 0.4252917127
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2382092	test: 9.2349274	best: 9.2349274 (0)	total: 6.94ms	remaining: 34.7s
4999:	learn: 0.3942015	test: 0.4297247	best: 0.4297247 (4999)	total: 32s	remaining: 0us
bestTest = 0.4297247332
bestIteration = 4

2it [07:03, 212.76s/it]

...

3it [10:32, 210.87s/it]

Learning rate set to 0.072699
0:	learn: 9.2399369	test: 9.2189216	best: 9.2189216 (0)	total: 7.67ms	remaining: 38.3s
4999:	learn: 0.4044326	test: 0.4377866	best: 0.4377866 (4999)	total: 29s	remaining: 0us
bestTest = 0.4377866061
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2328120	test: 9.2497528	best: 9.2497528 (0)	total: 7.63ms	remaining: 38.2s
4999:	learn: 0.4048932	test: 0.4395789	best: 0.4395789 (4999)	total: 27.6s	remaining: 0us
bestTest = 0.4395788932
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2306174	test: 9.2642683	best: 9.2642683 (0)	total: 7.14ms	remaining: 35.7s
4999:	learn: 0.4054556	test: 0.4385410	best: 0.4385410 (4999)	total: 26.7s	remaining: 0us
bestTest = 0.438541034
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2374137	test: 9.2350686	best: 9.2350686 (0)	total: 7.4ms	remaining: 37s
4999:	learn: 0.4037815	test: 0.4381566	best: 0.4381566 (4999)	total: 27.2s	remaining: 0us
bestTest = 0.4381566122
bestIteration = 499

4it [13:50, 205.94s/it]

Learning rate set to 0.072699
0:	learn: 9.2404424	test: 9.2208279	best: 9.2208279 (0)	total: 7.61ms	remaining: 38s
4999:	learn: 0.3884436	test: 0.4215482	best: 0.4215482 (4999)	total: 29.5s	remaining: 0us
bestTest = 0.4215482027
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2334973	test: 9.2508896	best: 9.2508896 (0)	total: 7.12ms	remaining: 35.6s
4999:	learn: 0.3882227	test: 0.4227087	best: 0.4227087 (4999)	total: 29.5s	remaining: 0us
bestTest = 0.4227087018
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2300984	test: 9.2639648	best: 9.2639648 (0)	total: 12.8ms	remaining: 1m 4s
4999:	learn: 0.3888803	test: 0.4226893	best: 0.4226893 (4999)	total: 29.2s	remaining: 0us
bestTest = 0.4226892564
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2368007	test: 9.2340004	best: 9.2340004 (0)	total: 7.16ms	remaining: 35.8s
4999:	learn: 0.3894633	test: 0.4231121	best: 0.4231121 (4999)	total: 29.7s	remaining: 0us
bestTest = 0.4231120583
bestIteration =

5it [17:28, 209.65s/it]


In [48]:
import pickle
with open('./train_deep.pkl', 'wb') as fout:
        pickle.dump(train_df[generate_feats], fout)

with open('./test_deep.pkl', 'wb') as fout:
    pickle.dump(test_df[generate_feats], fout)

with open('./list_features.pkl', 'wb') as fout:
    pickle.dump(list_features, fout)

In [49]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 64


In [60]:
X = n_train_df[remain_cols].values
Xtest = n_test_df[remain_cols].values
y = n_train_df[target[0]]

In [61]:
cat_models = train_model_by_X_y(X, y)

Learning rate set to 0.072699
0:	learn: 9.2454976	test: 9.2249389	best: 9.2249389 (0)	total: 7.57ms	remaining: 37.9s
4999:	learn: 0.3837964	test: 0.4185416	best: 0.4185416 (4999)	total: 34s	remaining: 0us
bestTest = 0.4185415674
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2373877	test: 9.2540063	best: 9.2540063 (0)	total: 10.2ms	remaining: 51.2s
4999:	learn: 0.3846511	test: 0.4208790	best: 0.4208790 (4999)	total: 37.1s	remaining: 0us
bestTest = 0.4208789724
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2317342	test: 9.2654284	best: 9.2654284 (0)	total: 7.75ms	remaining: 38.8s
4999:	learn: 0.3856827	test: 0.4196351	best: 0.4196351 (4999)	total: 33.9s	remaining: 0us
bestTest = 0.4196351183
bestIteration = 4999
Learning rate set to 0.072699
0:	learn: 9.2418356	test: 9.2387995	best: 9.2387995 (0)	total: 8.02ms	remaining: 40.1s
4999:	learn: 0.3845962	test: 0.4205554	best: 0.4205554 (4999)	total: 33.7s	remaining: 0us
bestTest = 0.420555399
bestIteration = 

In [64]:
preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(Xtest))
s = 0
for pred in preds:
    s += pred
s = s/5
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s * 0.05 + y_best * 0.95
df.to_csv('submission_corr_important_with_more.csv', index=False)



from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 39.02it/s]


cos_sim with best submission: 0.9936021707189874


In [47]:
s

array([28.59100774, 28.47203279, 27.73160931, ...,  0.97160002,
        0.4896039 ,  0.21563204])